In [1]:
import tensorflow_datasets as tfds

In [2]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteILNI28/imdb_reviews-train.tfrecord*...…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteILNI28/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteILNI28/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
print(info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset.
    This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_path='~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0',
    file_format=tfrecord,
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=25000, num_shards=1>,
        '

In [4]:
print(imdb)

{Split('train'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, Split('test'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, Split('unsupervised'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}


In [5]:
for ex in imdb["train"].take(2):
  print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on

In [6]:
import numpy as np

In [18]:
train_data, test_data = imdb["train"], imdb["test"]

In [36]:
training_sentences = []
training_labels = []

test_sentences = []
test_labels = []

for s,l in train_data:
  training_sentences.append(s.numpy().decode("utf8"))
  training_labels.append(l.numpy())

for s,l in test_data:
  test_sentences.append(s.numpy().decode("utf8"))
  test_labels.append(l.numpy())

In [37]:
training_labels = np.array(training_labels)
test_labels = np.array(test_labels)

In [21]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [28]:
vocab_size = 10000
max_length = 120
oov_token = "<OOV>"
trunc_type= "post"
embedding_dim = 16

In [38]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_idx = tokenizer.word_index

seqs = tokenizer.texts_to_sequences(training_sentences)
padded_seqs = pad_sequences(seqs, maxlen=max_length, truncating=trunc_type)

test_seqs = tokenizer.texts_to_sequences(test_sentences)
test_padded_seqs = pad_sequences(test_seqs, maxlen=max_length, truncating=trunc_type)

In [39]:
import tensorflow as tf
model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(6, activation="relu"),
                             tf.keras.layers.Dense(1, activation="sigmoid")])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 120, 16)           160000    
                                                                 
 flatten_6 (Flatten)         (None, 1920)              0         
                                                                 
 dense_12 (Dense)            (None, 6)                 11526     
                                                                 
 dense_13 (Dense)            (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.fit(padded_seqs, training_labels, epochs=10, validation_data=(test_padded_seqs, test_labels))

Epoch 1/10
782/782 [==============================] - 5s 6ms/step - loss: 0.5154 - accuracy: 0.7167 - val_loss: 0.3791 - val_accuracy: 0.8302
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2468 - accuracy: 0.9055 - val_loss: 0.4074 - val_accuracy: 0.8219
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0971 - accuracy: 0.9752 - val_loss: 0.5050 - val_accuracy: 0.8098
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0259 - accuracy: 0.9966 - val_loss: 0.6099 - val_accuracy: 0.8063
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0100 - accuracy: 0.9988 - val_loss: 0.6704 - val_accuracy: 0.8066
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0047 - accuracy: 0.9995 - val_loss: 0.7355 - val_accuracy: 0.8040
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.7831 - val_accuracy: 0.8077
Epoch 

In [44]:
# embedding layer
model.layers[0].get_weights()[0].shape

(10000, 16)

In [48]:
embedding_weights = model.layers[0].get_weights()[0]

In [50]:
reverse_word_index = tokenizer.index_word

In [46]:
import io

In [51]:
out_v = io.open("vecs.tsv", "w", encoding="utf-8")
out_m = io.open("meta.tsv", "w", encoding="utf-8")

for word_num in range(1, vocab_size):
  word_name = reverse_word_index[word_num]
  word_embedding = embedding_weights[word_num]
  out_m.write(word_name + "\n")
  out_v.write("\t".join([str(x) for x in word_embedding]) + "\n")

out_v.close()
out_m.close()